In [1]:
from merge_packages import merge_packages
import os
import requests
from tqdm.auto import tqdm

from github import Github
import json

from glob import glob
from bs4 import BeautifulSoup
import re

from urllib.parse import urlparse


In [2]:
cur_api = Github(os.environ['CLIENT_ID'], os.environ['CLIENT_SECRET'])
cur_api.rate_limiting[0]


5000

### Get Readmes

In [3]:
cur_packages = merge_packages()


In [19]:
for aa in cur_packages[cur_packages.type != "decibans"].name.tolist():
    print(aa)


AbaqusReader
AbstractAlgebra
AbstractDomains
AbstractFFTs
AbstractIndices
AbstractInstances
AbstractLattices
AbstractLogic
AbstractMCMC
AbstractNumbers
AbstractOperators
AbstractPlotting
AbstractRationals
AbstractWallets
Absynth
AcceleratedArrays
AccurateArithmetic
AcFDTD
ActiveAppearanceModels
ActuarialScience
AcuteML
Adapt
AdaptiveDistanceFields
AdaptiveRejectionSampling
ADCME
AdjacentFloats
AdmittanceModels
AdobeGlyphList
AdvancedHMC
adwaita_icon_theme_jll
AffineArithmetic
AffineInvariantMCMC
Agents
AgnosticBayesEnsemble
AhoCorasickAutomatons
AIBECS
AIControl
AiyagariContinuousTime
alg4opt_test
AlgebraicMultigrid
AlgebraicNumbers
AlgebraResultTypes
Algencan
Alistair
AllanDeviations
Alpaca
AlphaGo
AlphaVantage
Alpine
AltDistributions
Amb
AMDGPUnative
AMQPClient
Anamnesis
Anasol
AndersonMoore
AngleBetweenVectors
Animations
AnomalyDetection
ANOVA
Ant
APL
AppConf
ApplicationBuilder
ApplicationBuilderAppUtils
ApproxBayes
ApproxFunBase
ApproxFunFourier
ApproxFunOrthogonalPolynomials
Appro

OMETIFF
OMJulia
Omniscape
OmniSci
OnePhase
OnlineLearning
OnlinePackage
OnlineStatsBase
OOESAlgorithm
OOPMacro
OpenAIGym
OpenBLAS_jll
OpenDSSDirect
OpenFOAM
OpenIDConnect
OpenPixelControl
OpenSoundControl
OpenSpecFun_jll
OpenSSL_jll
OpenStreetMapPlotter
OpenStreetMapXPlot
OpenTrick
OpSel
OptimBase
OptimizationProblems
OptimPack
OptimPack_jll
OptimTestProblems
OptionalData
Optionals
Opus_jll
Oracle
ORCA
OrdinaryDiffEq
Org
OrthogonalPolynomialsQuasi
OSXNotifier
Overseer
p7zip_jll
PackageCompiler
PackageCompilerX
Packing
PaddleBattle
Pagerank
Paillier
PAINTER
PairwiseListMatrices
Pajarito
PandasLite
Pandoc
Pango_jll
ParallelProcessingTools
ParameterisedModule
ParameterizedFunctions
ParameterJuMP
Parametron
Parsers
ParSpMatVec
PartedArrays
ParticleMDI
ParticleScattering
PassiveTracerFlows
Patchelf_jll
PathDistribution
PATHlib_jll
PathLoss
patterns
Pavito
Pcap
PCL
PCRE2
PCRE2_jll
PCRE_jll
PdbTool
PDSampler
Peaks
Pedigrees
PEG
PenaltyFunctions
perceptron-jl
PeriodicTable
Permutations
Persa
P

In [5]:
query_params = "?" + "client_id=" + os.environ['CLIENT_ID']
query_params += "&" + "client_secret=" + os.environ['CLIENT_SECRET']

for cur_row in tqdm(cur_packages.itertuples(), total=len(cur_packages)):
    
    cur_name = f"tmp/html_readmes/{cur_row.name}.txt"
    
    if os.path.exists(cur_name) : continue
    
    cur_url = cur_row.url.replace("github.com", "api.github.com/repos")
    cur_url += f"/contents/README.md{query_params}"

    cur_response = requests.get(
        cur_url, headers = { "Accept": "application/vnd.github.html" }
    )
    
    with open(cur_name, "w") as cur_file:
        cur_file.write(cur_response.text)
        

### Get Homepages

In [10]:
with open('data/homepages.json') as cur_file:
    cur_json = json.load(cur_file)

In [27]:
for cur_row in tqdm(cur_packages.itertuples(), total=len(cur_packages)):
    cur_name = cur_row.name
    if cur_name in cur_json : 
        print(404)
        continue
        
    cur_path = "/".join(cur_row.url.split("/")[-2:])

    cur_homepage = None
    
    try:
        cur_homepage = cur_api.get_repo(cur_path).homepage
    except:
        continue
        
    if cur_homepage == None : continue
        
    cur_homepage = cur_homepage.strip().strip("/")
        
    cur_json[cur_name] = cur_homepage


In [36]:
with open('data/homepages.json', 'w') as cur_file:
    json.dump(cur_json, cur_file)
    

### Digest Readmes

In [57]:
for cur_file in tqdm(glob("tmp/html_readmes/*")):
    raw_readme = open(cur_file).read()
    cur_soup = BeautifulSoup(raw_readme, 'html.parser')

    cur_remove_list = []
    cur_remove_list.extend(
        cur_soup.findAll('div', {"class": "highlight"})
    )

    cur_remove_list.extend(cur_soup.findAll('pre'))
    cur_remove_list.extend(cur_soup.findAll('code'))

    for cur_remove_item in cur_remove_list:
        cur_remove_item.decompose()

    cur_text = re.sub("\n\n+", "\n\n", cur_soup.text)
    cur_name = cur_file.replace("/html_readmes/", "/text_readmes/")
    
    with open(cur_name, "w") as cur_file:
        cur_file.write(cur_text)


In [63]:
ignored_domains = [
    "travis-ci.com",
    "travis-ci.org",
    "codecov.io",
    "coveralls.io",
    "appveyor.com",
    "gitter.im",
    "reddit.com",
    "vimeo.com",
    "xkcd.com",
    "amazon.com",
    "wikipedia.org",
    "github.com",
    "githubusercontent.com",
    "discourse.julialang.org",
    "pkg.julialang.org",
    "gitlab.com",
    "bitbucket.org",
    "doi.org",
    "arxiv.org"
]

ignored_filetypes = [
    ".png",
    ".jpg",
    ".jpeg",
    ".pdf",
    ".gif",
    ".ipynb"
]

anchor_dict = {}

for cur_file in tqdm(glob("tmp/html_readmes/*")):
    cur_package = cur_file.replace(".txt", "").split("/")[-1]
    
    raw_readme = open(cur_file).read()
    cur_soup = BeautifulSoup(raw_readme, 'html.parser')
    
    cur_anchors = map(
        lambda cur_anchor: cur_anchor.get("href"), 
        cur_soup.select("a:not(.anchor)")
    )
    
    cur_anchors = filter(None, cur_anchors)
    
    cur_anchors = [
        cur_anchor for cur_anchor in cur_anchors
        if not any(map(lambda ignored_domain: ignored_domain in cur_anchor, ignored_domains))
    ]
    
    cur_anchors = [
        cur_anchor for cur_anchor in cur_anchors
        if not any(map(lambda ignored_filetype: cur_anchor.endswith(ignored_filetype), ignored_filetypes))
    ]
    
    cur_anchors = [
        cur_anchor.split("#")[0].split("?")[0].strip("/") for cur_anchor in cur_anchors
        if "http" in cur_anchor
    ]
    
    cur_anchors = list(sorted(set(cur_anchors)))
    
    if len(cur_anchors) == 0 : continue

    anchor_dict[cur_package] = cur_anchors


In [93]:
ignored_urls = [
    "http://julialang.org",
    "https://julialang.org",
    "http://www.repostatus.org",
    "https://www.repostatus.org",
    "http://julialang.org/downloads",
    "https://julialang.org/downloads",
    "http://www.juliacomputing.com",
    "https://www.juliacomputing.com",
    "http://www.julialang.org",
    "https://www.julialang.org"
]

for cur_key, cur_value in anchor_dict.items():
    tmp_value = []
    
    for work_value in cur_value:
        if work_value in ignored_urls : continue
            
        if ".zip" in work_value : continue
        if "cirrus-ci.com" in work_value : continue
            
        parsed_url = urlparse(work_value)
        if parsed_url.netloc == "" : continue
        
        if not ( parsed_url.scheme == "http" or parsed_url.scheme == "https" ) :
            continue
            
        tmp_value.append(parsed_url.geturl())

    anchor_dict[cur_key] = list(set(tmp_value))
    

In [94]:
with open('data/anchors.json', 'w') as cur_file:
    json.dump(anchor_dict, cur_file)
    